In [9]:
import os
from os import environ
from typing import cast

from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_core.language_models import LanguageModelInput
from langchain_core.messages import BaseMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_gigachat.chat_models import GigaChat
from langchain_gigachat.embeddings import GigaChatEmbeddings

### Examples


#### init


Установка `tagme_gigachain`

In [17]:
%pip install ..

Processing /Users/red/projects/gigachain/cookbook/tagme_gigachain
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tagme_gigachain: filename=tagme_gigachain-0.1.5-py3-none-any.whl size=17191 sha256=3c9ad2693d481c889a603ffa496457c855e61a6b7129404078b5d79fd6f60f37
  Stored in directory: /private/var/folders/vc/l2_j90hx25zc34c03s2v566c0000gn/T/pip-ephem-wheel-cache-qskeogmy/wheels/71/71/59/6de7c2f0150ef0006263d88a8bc02a175bd2f38c0568b8f4f6
Successfully built tagme_gigachain
  Attempting uninstall: tagme_gigachain
    Found existing installation: tagme_gigachain 0.1.5
    Uninstalling tagme_gigachain-0.1.5:
      Successfully uninstalled tagme_gigachain-0.1.5
Note: you may need to restart the kernel to use updated packages.


Для работы нужны переменные TAGME_GIGACHAIN_TOKEN и TAGME_GIGACHAIN_BASE_URL. Клиент автоматически поднянет их. Или можно указать напрямую в kwargs клиентов или декораторов

In [ ]:
token = os.environ.get("TAGME_GIGACHAIN_TOKEN",
                       "0a3618fa-97a2-404a-b95d-15d0c3a6fa4d")
base_url = os.environ.get(
    "TAGME_GIGACHAIN_BASE_URL", "https://tagme.sberdevices.ru/dev/chatwm/plugin_statistics/trace"
)

In [11]:
%set_env TAGME_GIGACHAIN_TOKEN=0a3618fa-97a2-404a-b95d-15d0c3a6fa4d
%set_env TAGME_GIGACHAIN_BASE_URL=https://tagme.sberdevices.ru/dev/chatwm/plugin_statistics/trace

env: TAGME_GIGACHAIN_TOKEN=0a3618fa-97a2-404a-b95d-15d0c3a6fa4d
env: TAGME_GIGACHAIN_BASE_URL=https://tagme.sberdevices.ru/dev/chatwm/plugin_statistics/trace


#### Использование декораторов на примере пойплайна по генерации ответа модели с использованием RAG


##### Формируем пайплайн

In [12]:
# Создание документов
documents = [
    Document(page_content="Собаки — отличные компаньоны, известные своей преданностью."),
    Document(page_content="Кошки — независимые животные, которым нужно собственное пространство."),
    Document(page_content="Попугаи — умные птицы, способные имитировать человеческую речь."),
]

# Инициализация модели эмбеддингов
embeddings = GigaChatEmbeddings(
    credentials=environ.get("GIGA_TOKEN"),
    # base_url="https://gigachat.devices.sberbank.ru/api",
    # auth_url="https://ngw.devices.sberbank.ru:9443/api",
    verify_ssl_certs=False,
    scope="GIGACHAT_API_CORP",
)

# Создание векторного хранилища
vectorstore = Chroma.from_documents(documents, embeddings)


retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)  # выбор наиболе подходящего результата

retriever.batch(["кошка", "акула"])

[[Document(id='076cec87-ee8d-46ae-aebc-e0acb919d3ee', metadata={}, page_content='Кошки — независимые животные, которым нужно собственное пространство.')],
 [Document(id='c9b36384-0e49-41fd-942c-5b23cd7a567c', metadata={}, page_content='Собаки — отличные компаньоны, известные своей преданностью.')]]

In [13]:
model = GigaChat(
    credentials=environ.get("GIGA_TOKEN"),
    verify_ssl_certs=False,
    scope="GIGACHAT_API_CORP",
)

In [14]:
message = """
Отвечай на вопросы только с помощью полученного контекста.

{question}

Контекст:
{context}
"""

prompt = ChatPromptTemplate.from_messages(
    [("human", message), ("assistant", "не могу тебе помочь"), ("human", "почему?")]
)

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | model


##### Асинхронный декоратор


In [ ]:
from tagme_gigachain import tagme_trace_async


@tagme_trace_async(token=token, metadata={"experiment": "#1"}, base_url=base_url)
async def run_model_async(input: LanguageModelInput) -> BaseMessage:
    return await model.ainvoke(input)


rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | run_model_async

response = cast(BaseMessage, await rag_chain.ainvoke("Опиши кошек двумя словами"))

response.content

##### Синхронный декоратор

In [ ]:
from tagme_gigachain import tagme_trace


@tagme_trace(token=token, metadata={"model": "GigaChat-2"})
def run_model_sync(input: LanguageModelInput) -> BaseMessage:
    return model.invoke(input)


rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | run_model_sync

response = rag_chain.invoke("Опиши кошек одним словом")

response.content

'Потому что в представленном контексте содержится лишь информация о кошках как о независимых животных, нуждающихся в собственном пространстве. Однако одного слова недостаточно, чтобы обобщённо описать характер и поведение всех кошек. В данном случае использование единственного термина приведёт к упрощению и искажению восприятия кошек как живых существ. Если нужно конкретное слово, попробуй уточнить вопрос, указав направление описания (например: ласковые, игривые, домашние, дикие).'

#### Прямой запрос


Инициализируем клиент

In [30]:
from tagme_gigachain import TagmeIntegrationClientAsync, TagmeIntegrationClientSync

In [ ]:
tagme_trace_client = TagmeIntegrationClientAsync(
    token=token,  # передаём токен или используем env TAGME_GIGACHAIN_TOKEN
    base_url=None,  # передаём base_url или используем env TAGME_GIGACHAIN_BASE_URL
    ignore_missing_functions=True  # не вызываеть ошибки, если в диалоге содержится функция, описание которой заранее не было отправлено на сервер
)

##### Отправка диалога


In [ ]:
from tagme_gigachain.entities import DialogData, ChatMessage


dialog_delete_alarm = DialogData(
    input=[
        ChatMessage.from_dict({"role": "system", "content": "You are a smart home assistant."}),
        ChatMessage.from_dict({"role": "user", "content": "Удалите будильник на 7 утра."}),
        ChatMessage.from_dict(
            {
                "role": "assistant",
                "content": None,
                "function_call": {"name": "delete_alarm_unknown", "arguments": {"time": "07:00"}},
            }
        ),
        ChatMessage.from_dict(
            {
                "role": "function",
                "content": None,
                "function_result": {"name": "delete_alarm_unknown", "result": {"status": "success"}},
            }
        ),
        ChatMessage.from_dict({"role": "assistant", "content": "Будильник на 7:00 удалён."}),
    ],
    metadata={"test": "delete_alarm_unknown"},
)


In [32]:
await tagme_trace_client.send_dialog(dialog_delete_alarm)

{'id': '8e7ceea8-7960-4e30-b85a-f83624831347',
 'missing': ['delete_alarm_unknown_2'],
 'linked_functions': [],
 'warning': True}

##### Отравка описания функции на сервер


In [ ]:
function_delete_alarm = {
    "name": "delete_alarm_unknown",
    "description": "Функция удаления будильника по id. Данная функция запускается только при наличии необходимых идентификаторов id будильников в контексте. Если пользователь явно не передал id будильника, то получи метаинформацию об установленных будильниках, вызвав сначала соответствую функцию и только затем используй функцию удаления по id. Если пользователь просит удалить все будильники и в контексте диалога есть необходимые id или пользователь явно передает id будильника, который надо удалить, то вызови эту функцию, переспрашивать пользователя не нужно. В остальных случаях, при наличии необходимых id в контексте диалога и готовности удалить будильник, сначала переспроси пользователя подтверждает ли он удаление будильника и вызывай функцию только при наличии подтверждения от пользователя.",
    "parameters": {
        "properties": {
            "ids": {
                "description": "Список id будильников, которые нужно удалить",
                "items": {"description": "Идентификатор id будильника, который нужно удалить", "type": "string"},
                "type": "array",
            }
        },
        "required": ["ids"],
        "type": "object",
    },
    "return_parameters": {
        "description": "Ответ на delete_alarm",
        "properties": {
            "error": {"description": "Текст ошибки в случае, если status == fail", "type": "string"},
            "ids": {
                "description": "Список id будильников, которые удалились",
                "items": {"description": "Идентификатор id будильника, который удалился", "type": "string"},
                "type": "array",
            },
            "status": {
                "description": "Статус - удалось ли удалить будильник.",
                "type": "string",
                "enum": ["success", "fail"],
            },
        },
        "required": ["status"],
        "type": "object",
    },
    "few_shot_examples": [],
}


In [17]:
from tagme_gigachain.entities import FunctionDef


await tagme_trace_client.send_functions(functions=[FunctionDef.from_dict(function_delete_alarm)])

{'results': [{'name': 'delete_alarm_unknown',
   'id': 4,
   'version': 1,
   'status': 'skipped'}]}

In [18]:
await tagme_trace_client.get_functions()

[FunctionResponse(id=1, name='delete_alarm2', version=1, definition={'description': 'Функция удаления будильника по id. Данная функция запускается только при наличии необходимых идентификаторов id будильников в контексте. Если пользователь явно не передал id будильника, то получи метаинформацию об установленных будильниках, вызвав сначала соответствую функцию и только затем используй функцию удаления по id. Если пользователь просит удалить все будильники и в контексте диалога есть необходимые id или пользователь явно передает id будильника, который надо удалить, то вызови эту функцию, переспрашивать пользователя не нужно. В остальных случаях, при наличии необходимых id в контексте диалога и готовности удалить будильник, сначала переспроси пользователя подтверждает ли он удаление будильника и вызывай функцию только при наличии подтверждения от пользователя.', 'parameters': {'properties': {'ids': {'description': 'Список id будильников, которые нужно удалить', 'items': {'description': 'Ид

Уже не будет missing - описание функции подвязалось с использованной в диалоге и будет отправляться разметчикам


In [19]:
await tagme_trace_client.send_dialog(dialog_delete_alarm)

{'id': '4829e51c-c948-41df-9322-5e700b0fe728',
 'missing': [],
 'linked_functions': [{'id': 4, 'name': 'delete_alarm_unknown', 'version': 1}],
 'warning': False}

#### Получение результатов и статистики разметки


In [20]:
await tagme_trace_client.get_results()

[]

In [21]:
await tagme_trace_client.get_markup_quality()

[]

In [22]:
await tagme_trace_client.get_markup_statistics()

{'task_id': '656d8511-fc9a-4d17-b608-8094b6f6ac23',
 'total_dialogs': 49,
 'marked_dialogs': 0,
 'total_markups': 0,
 'accepted_markups': 0,
 'overlap': 1}